<a href="https://colab.research.google.com/github/conwayjw97/Music-Generator-LSTM/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Tutorial Link](https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5)

# Library Imports

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import glob
import numpy as np

from music21 import converter, instrument, note, chord
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Input
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.utils import to_categorical

# Dataset Download

In [2]:
# Initialise empty repository to pull data into
!git init 
# Add the remote origin
!git remote add origin -f https://github.com/Skuldur/Classical-Piano-Composer.git
# Reset the HEAD in case a different folder was already pulled
!git reset --hard HEAD
!git clean -f -d
# Tell git we are checking out specific folders
!git config core.sparsecheckout true
# Recursively checkout the needed folders
!echo "data/*" >> .git/info/sparse-checkout
!echo "midi_songs/*" >> .git/info/sparse-checkout
!echo "weights.hdf5" >> .git/info/sparse-checkout
# Pull dataset from repositoryF
!git pull origin master

Initialized empty Git repository in /content/.git/
Updating origin
remote: Enumerating objects: 334, done.
remote: Total 334 (delta 0), reused 0 (delta 0), pack-reused 334
Receiving objects: 100% (334/334), 721.79 MiB | 40.53 MiB/s, done.
Resolving deltas: 100% (41/41), done.
From https://github.com/Skuldur/Classical-Piano-Composer
 * [new branch]      adding_support_for_files_with_no_parts -> origin/adding_support_for_files_with_no_parts
 * [new branch]      apply-pylint-to-codebase -> origin/apply-pylint-to-codebase
 * [new branch]      master               -> origin/master
 * [new branch]      no-data              -> origin/no-data
 * [new branch]      offsets_and_duration -> origin/offsets_and_duration
 * [new branch]      revert-19-master     -> origin/revert-19-master
 * [new branch]      varying_speed_notes  -> origin/varying_speed_notes
fatal: ambiguous argument 'HEAD': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git 

# Dataset Preparation

The data is split into two objects. 

Notes, that contain information about:
* Pitch: Frequency of the sound
* Octave: Set of pitches
* Offset: Where the note is located

Chords, which are containers for a set of notes played at the same time

In [0]:
# Load the data into an array
notes = []
for file in glob.glob("midi_songs/*.mid"):

    # Load file into a Music21 stream object
    midi = converter.parse(file)

    # Get a list of all notes and chords in the file
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # File has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # File has notes in a flat structure
        notes_to_parse = midi.flat.notes

    # Append the pitch of every note object using its string notation
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

# Get amount of pitch names
n_vocab = len(set(notes))

# Each sequence can be this many notes/chords, to predict the next note in the sequence
# the network will use this many previous notes to make the prediction. Worth trying
# different sequence lengths
sequence_length = 100

# Get all pitch names
pitchnames = sorted(set(item for item in notes))

# Map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []

# Create input sequences and corresponding outputs for the network
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

# Reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

# Normalise input
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)

# LSTM Definition

In [0]:
# Input layer
input = Input(shape=(network_input.shape[1], network_input.shape[2]))

# Hidden layers
layer = LSTM(256, return_sequences=True)(input)
layer = Dropout(0.3)(layer)
layer = LSTM(512, return_sequences=True)(layer)
layer = Dropout(0.3)(layer)
layer = LSTM(256)(layer)
layer = Dense(256)(layer)
layer = Dropout(0.3)(layer)
layer = Dense(n_vocab)(layer)

# Output layer
output = Activation('softmax')(layer)

# Compile model
lstm = Model(input, output, name='lstm')
lstm.compile(loss='categorical_crossentropy', optimizer='rmsprop')
lstm.summary()

# Load the weights to each node
# lstm.load_weights('weights.hdf5')

Model: "lstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100, 1)]          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792  

# Train Model

In [0]:
history = lstm.fit(network_input, network_output, epochs=50, batch_size=64)

Epoch 1/50
892/892 [==============================] - 115s 129ms/step - loss: 4.7091
Epoch 2/50
652/892 [====================>.........] - ETA: 30s - loss: 4.7086

# Generate Music